In [ ]:
!pip install spotipy

In [22]:
import spotipy  
from spotipy.oauth2 import SpotifyClientCredentials,SpotifyOAuth

In [ ]:
CLIENT_ID ="e698f01eb31c44359db77e4c19b1ddd4"
CLIENT_SECRET="27a45297ef9144758f5174454965bf73"
REDIRECT_URI="https://localhost:3000"
SCOPE="user-library-read"

client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

#refresh_token = sp_oauth.get_refresh_token()  

In [21]:
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=CLIENT_ID,
                                               client_secret=CLIENT_SECRET,
                                               redirect_uri=REDIRECT_URI,
                                               scope=SCOPE))

In [ ]:
sp.search()

In [17]:
birdy_uri = 'spotify:artist:2WX2uTcsvV5OnS0inACecP'
results = sp.artist_albums(birdy_uri, album_type='album')
albums = results['items']
while results['next']:
    results = sp.next(results)
    albums.extend(results['items'])

for album in albums:
    print(album['name'])

Portraits (+ Remixes)
Portraits
Young Heart
Beautiful Lies (Deluxe)
Beautiful Lies
Fire Within (Deluxe)
Live in London
Birdy (Deluxe Version)
Birdy


In [ ]:
birdy_uri = 'spotify:artist:2WX2uTcsvV5OnS0inACecP'
results = sp.artist_albums(birdy_uri, album_type='album')

## Discogs

In [25]:
import discogs_client
d = discogs_client.Client('jazzmusicians/0.1',user_token="LFMgTmNktxNIMJwRHfeZobKBvFEYAHpPtMyroiMW")


In [49]:

r = d.search('genre=jazz',type='release')

In [ ]:

results = d.search('',type='release',genre="jazz")
print(results.pages)
counter = 0
for i in range(1,results.pages) :
    counter+=len(results.page(i))
print(counter)    

In [4]:
class Album(object):
  def __init__(self,title,artists_name,artists_id,countries,year,labels,labels_id):
    self.title = title
    self.artists_name = artists_name
    self.artists_id = artists_id
    self.countries= countries
    self.year= year
    self.labels= labels
    self.labels_id=labels_id

  def to_dict(self):
        return {
            'title': self.title,
            'artists_name': self.artists_name,
            'artists_id' :self.artists_id,
            'countries' : self.countries,
            'year' : self.year,
            'labels' : self.labels,
            'labels_id' : self.labels_id
        }   

In [3]:
def get_artists_info(data) :
 artists_name = list()
 artists_id = list()
 for i in data : 
   artists_name.append(i['name'])
   artists_id.append(i['id'])

 return (artists_name,artists_id)

def get_labels_info(data) :
 labels_name = list()
 #labels_id = list()
 print(data)
 for i in data : 
   labels_name.append(i['name'])
   #labels_id.append(i['id'])

 return labels_name

In [12]:
import requests
import time

In [103]:
response =  requests.get("https://api.discogs.com/releases/"+"11357739",headers= {"Authorization":"Discogs token=LFMgTmNktxNIMJwRHfeZobKBvFEYAHpPtMyroiMW"}) 
#print(response.json()['results'][3]["id"])
response.headers['X-Discogs-Ratelimit-Remaining']

'60'

In [ ]:
response = requests.get("https://api.discogs.com/database/search?genre=jazz&type=release&format=album&per_page=100&token=LFMgTmNktxNIMJwRHfeZobKBvFEYAHpPtMyroiMW")
albums_json = response.json()['results']
albums_json[1]

In [5]:
response = requests.get("https://api.discogs.com/database/search?genre=jazz&type=release&format=album&per_page=100&token=LFMgTmNktxNIMJwRHfeZobKBvFEYAHpPtMyroiMW")
nbPages = response.json()['pagination']['pages']
albums = list()
for page in range(2,nbPages):
    albums_json = response.json()['results']
    for i in range(1,len(albums_json)) :
        #print(albums_json[i])
        #labels_info = get_labels_info(albums_json[i]['label'])
        album = Album(albums_json[i]['title'],'','',albums_json[i]['country'],'',albums_json[i]['label'],'')
        albums.append(album)
    
    if int(response.headers['X-Discogs-Ratelimit-Remaining']) == 1:# limit rate : 60 requests/min 
            print("SLEEP...")
            time.sleep(60)    
    response = requests.get("https://api.discogs.com/database/search?genre=jazz&type=release&format=album&page="+ str(page) +"&per_page=100&token=LFMgTmNktxNIMJwRHfeZobKBvFEYAHpPtMyroiMW")

SLEEP...


In [6]:
len(albums)

9702

In [7]:
import numpy as np
import pandas as pd
df = pd.DataFrame.from_records([a.to_dict() for a in albums])

In [8]:
df

,title,artists_name,artists_id,countries,year,labels,labels_id
0,Gregory Porter - One Night Only (Live At The R...,,,Europe,,"[Blue Note, Eagle Rock Films, Decca Records Fr...",
1,Brother Ah* - Divine Music,,,US,,[Manufactured Recordings],
2,The Wynton Marsalis Quintet & Richard Galliano...,,,US,,[Rampart Street Records],
3,Jim Hall & Red Mitchell - Valse Hot - Sweet Ba...,,,US,,"[ArtistShare, Sweet Basil, Sterling Sound]",
4,Nah Youn Sun* = 나윤선* - Memory Lane,,,South Korea,,"[Seoul Records, Hub Music. Inc., Seoul Records...",
...,...,...,...,...,...,...,...
9697,Mike Baggetta - Canto,,,US,,"[Mabnotes Music, Mabnotes Music, Disc Makers]",
9698,Myriam Alter - If,,,Japan,,[Enja Records],
9699,"Michal Bugala Feat. Seamus Blake, Danny Crisse...",,,Slovakia,,[Hudobný Fond],
9700,"Gerry Mulligan - If You Can't Beat 'Em, Join 'Em!",,,UK,,[Mercury],


# Get albums information + credits : 


In [1]:
import musicBrainz.Album
import pandas as pd
import json
import requests
import time

In [2]:
df = pd.read_csv("rym_charts_part1.csv",sep=";")#O do 
albums = df["Album"]
artists = df["Artist"]
dates= df["Date"]

In [3]:
#for all the release we compute the sum of the want and have metric
def find_best_release_version(releases) :
    max_interest = 0 
    choosen_release = None
    for r in releases :
       sum =  int(r["community"]["have"]) +  int(r["community"]["want"])
       if sum >= max_interest:
           max_interest = sum
           choosen_release = r
    return choosen_release

In [5]:
release_json=[]
release_detail_json=[]
cnt_album_done = 0
start_time = time.time()
for idx in range(0,len(albums)) :
     
    #year = dates[idx].split()[1]
    release_response = requests.get("https://api.discogs.com/database/search?artist="+ artists[idx]  +"&release_title="+ albums[idx] +"&type=release&per_page=100&token=LFMgTmNktxNIMJwRHfeZobKBvFEYAHpPtMyroiMW")
    print( release_response)
    release_result = release_response.json()["results"]
    if len(release_result) == 0 :
        unknown_release = artists[idx] +"-"+ albums[idx] +"\n"
        with open("missing_releases.txt", "a",encoding='utf-8') as myfile:
            myfile.write(unknown_release)
        myfile.close()
        continue
    
    #TO DO : look for the want/have ratio to choose the best release version 
    release_result = find_best_release_version(release_result)

    release_json.append(release_result) 
    
    response = requests.get(release_result["resource_url"])
    release_detail_json.append(response.json())
    
    cnt_album_done+=1
     
    current_time = time.time() - start_time
    if int(response.headers['X-Discogs-Ratelimit-Remaining']) == 0 and  current_time < 60:
        sleeping_time = 60 - current_time
        print("Sleeping for ", sleeping_time, "secondes | Number of album Treated:",cnt_album_done)
        time.sleep(sleeping_time) 
        start_time = time.time()

    if current_time >=60 :  
       start_time = time.time()

with open('release_info2.json', 'w', encoding='utf-8') as f:
    json.dump(release_json, f, ensure_ascii=False, indent=4)
with open('release_details2.json', 'w', encoding='utf-8') as f:
    json.dump(release_detail_json, f, ensure_ascii=False, indent=4)

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
Sleeping for  29.047914266586304 secondes | Number of album Treated: 13
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
Sleeping for  52.61143159866333 secondes | Number of album Treated: 20
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
Sleeping for  53.27905488014221 secondes | Number of album Treated: 30
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
Sleeping for  55.674867391586304 secondes | Number of album Treated: 38


In [7]:
len(release_json)

456

# Examples 


In [ ]:
#try:
response = requests.get("https://api.discogs.com/database/search?genre=jazz&type=release&format=album&per_page=100&token=LFMgTmNktxNIMJwRHfeZobKBvFEYAHpPtMyroiMW")
nbPages = response.json()['pagination']['pages']
for page in range(2,nbPages):
    #print(response.json()['results'][3]["id"])
    albums_json = response.json()['results']
    albums = list()
    print("page=",len(albums_json))
    for i in range(1,len(albums_json)) :  
        album_id = albums_json[i]["id"]
    
        response = requests.get("https://api.discogs.com/releases/"+ str(album_id),headers= {"Authorization":"Discogs token=LFMgTmNktxNIMJwRHfeZobKBvFEYAHpPtMyroiMW"}) 
        data = response.json()
        if int(response.headers['X-Discogs-Ratelimit-Remaining']) == 1:
            print(data)
    
        #print(get_artists_info(data['artists']))
        artists_info = get_artists_info(data['artists'])
        labels_info = get_labels_info(data['labels'])
        #print(data['country'])
        album = Album(data['title'],artists_info[0],artists_info[1],data['country'],data['year'],labels_info[0],labels_info[1])
        albums.append(album)
        print('raimaing: ',response.headers['X-Discogs-Ratelimit-Remaining'])
        if int(response.headers['X-Discogs-Ratelimit-Remaining']) == 1:# limit rate : 60 requests/min 
            print("SLEEP...")
            time.sleep(1)
    print(page)
    response = requests.get("https://api.discogs.com/database/search?genre=jazz&type=release&format=album&page="+ str(page) +"&per_page=100&token=LFMgTmNktxNIMJwRHfeZobKBvFEYAHpPtMyroiMW")
#except:
 #     print(response) 
  #    print(data)
   #   print(data['artists'])
    #  print(get_artists_info(data['artists']))     

In [105]:
len(albums)


49

In [9]:
import xml.etree.ElementTree as ET
context = ET.iterparse("discogs_20231001_masters.xml", events=("start", "end"))
context

<xml.etree.ElementTree.iterparse.<locals>.IterParseIterator at 0x2a55dda47c0>

In [ ]:
from lxml import etree 
doc = etree.parse()
root = doc.getroot()

result = len(root.getchildren())

In [ ]:
import xmltodict, json
o = xmltodict.parse('discogs_20231001_masters.xml')
json.dumps(o) # '{"e": {"a": ["text", "text"]}}'